## Install python packages

In [9]:
%%bash

pip3 install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.2 MB/s eta 0:00:00


## Install the custom pyspark package

In [1]:
%%bash
./install-pyspark.sh

Processing ./pyspark-3.5.0.dev0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 5.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0.dev0-py2.py3-none-any.whl size=258907706 sha256=7adc3d8ebd06305f39744179fce878e68b98ca73091e26bb76fa641d501bca2c
  Stored in directory: /root/.cache/pip/wheels/09/88/fa/a22881c4ebc798868daf7cf5183c18d7081e6dc2ac0d0befd1
Successfully built pyspark


## Create the DB users

In [2]:
%%bash

export PGPASSWORD=postgres

create_db() {
cat <<EOF
        create user $1 with password '$1';
        create database $1;
        grant all privileges on database $1 to $1;
        \c $1 postgres
        grant all on schema public to $1;
EOF
}

dbnames=(lsqb snap tpcds tpch)

psql --host=postgres --username postgres --dbname postgres <<-EOSQL
        $(create_db lsqb)
        $(create_db snap)
        $(create_db tpcds)
        $(create_db tpch)
EOSQL

ERROR:  role "lsqb" already exists
ERROR:  database "lsqb" already exists


GRANT
You are now connected to database "lsqb" as user "postgres".
GRANT


ERROR:  role "snap" already exists
ERROR:  database "snap" already exists


GRANT
You are now connected to database "snap" as user "postgres".
GRANT


ERROR:  role "tpcds" already exists
ERROR:  database "tpcds" already exists


GRANT
You are now connected to database "tpcds" as user "postgres".
GRANT


ERROR:  role "tpch" already exists
ERROR:  database "tpch" already exists


GRANT
You are now connected to database "tpch" as user "postgres".
GRANT


## Fetch LSQB data

In [7]:
%%bash
cd lsqb
export MAX_SF=300
./scripts/download-merged-fk-data-sets.sh
cd ..

Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.14).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


## Import the LSQB benchmark data

In [11]:
%%bash
export SF=30
./import-lsqb.sh

DROP VIEW
DROP VIEW
DROP VIEW
DROP VIEW
DROP VIEW
DROP VIEW
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
DROP TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 1575
COPY 6380
COPY 6
COPY 111
COPY 1343
COPY 1831640
COPY 77110476
COPY 24025658
COPY 184000
COPY 16080
COPY 71
COPY 96662059
COPY 25336957
COPY 105337003
COPY 5937610
COPY 4289970
COPY 68979133
COPY 30729218
COPY 147243
COPY 400460
COPY 7273036
COPY 7273036
CREATE VIEW
CREATE VIEW
CREATE VIEW
CREATE VIEW
CREATE VIEW
CREATE VIEW


## Fetch the SNAP data

In [14]:
%%bash

cd snap
curl -O http://snap.stanford.edu/data/cit-Patents.txt.gz
gunzip cit-Patents.txt.gz
./remove-header.sh
cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81.1M  100 81.1M    0     0  5427k      0  0:00:15  0:00:15 --:--:-- 12.8M


## Import the SNAP data

In [15]:
%%bash

PGPASSWORD=snap psql -h postgres -U snap -d snap -f import-snap.sql

psql:import-snap.sql:1: NOTICE:  table "patents" does not exist, skipping


DROP TABLE
CREATE TABLE
COPY 16518948


## Generate the TPC-H data

In [ ]:
%%bash

export SF=100

cd tpch-kit/dbgen

yes y | ./dbgen -s $SF

mkdir -p data-$SF

mv *.tbl data-$SF/

cd ../..

TPC-H Population Generator (Version 2.17.3)
Copyright Transaction Processing Performance Council 1994 - 2010


## Import the TPC-H data

In [ ]:
%%bash

export SF=50

PGPASSWORD=tpch psql -h postgres -U tpch -f tpch-drop.sql

tables=$( ls tpch-kit/dbgen/data-$SF/*.tbl | xargs -n 1 basename | sed 's/.tbl$//')

for t in $tables; do
        PGPASSWORD=tpch psql -h postgres -U tpch -c "\copy $t from 'tpch-kit/dbgen/data-$SF/${t}.tbl' DELIMITER '|';"
done
